# CSCI 3202: Mini Project # 1

The goal of this mini-project is to implement "curve fitting" or regression for given data using __genetic algorithms__ (evolutionary search) and run various benchmark examples to study its performance. Optionally, you could with the other approach of simulated annealing. We will provide you all the necessary infrastructure for solving the problem. 

**TODO:** Program the genetic algorithm using the high-level methodology described here (and described in  the AIMA textbook). Please use the infrastructure provided.

**Submission:** A single zip file containing your
  - Python code: must be written by yourself. Please be careful not to copy from other students or online sources. We are aware of other projects and will do comparisons with other student's code. However, collaborations exchanging useful ideas and tips  with other students are highly encouraged.
  - `MiniProjectReportTemplate.ipynb`: fill out the notebook by answering the questions and having the benchmark results table there.


## Regression

The regression problem is a basic problem of finding a function $y = f(x)$ that passes through some given data points given as 

$$ (x_1, y_1), (x_2, y_2), \ldots, (x_n, y_n)$$

Regression is a fundamental problem in machine learning. However, since it is often hard and undesirable to find an exact fit (look up <a href="https://en.wikipedia.org/wiki/Overfitting"> overfitting </a> ), we aim to find a least square fit. I.e, find a function that minimizes the loss function.

$$ \text{min}\; ( (y_1 - f(x_1))^2 + (y_2 - f(x_2))^2 + \cdots + (y_n - f(x_n))^2 ) $$.

The main problem is that there is no idea what the form of $f$ looks like. Therefore, we need to search for $f$ from a family of functions $\{ f_1, \ldots, f_N \} $. Often, the set of functions in this family is not finite.

**Example:** linear regression looks for functions of the form $f: c x+ d$ for some unknowns $c, d$. This is already an infinite family of functions. The problem is complicated for nonlinear functions since we do not know what kind of nonlinear function to use.

In this mini-project, we will explore the use of genetic algorithms to search for a function $f$ from among many possibilities to find a function that fits a given set of data points as closely as possibly while making the least-squares loss as small as possible.

### Example 1
In the figure below: the green 'x' denote the data points $(x_i, y_i)$ and we know secretly that these points have been generated by the function $e^{\sin(2x)} - e^{\cos(3x)}$. However, the regression algorithm just sees the data samples. It's goal is to fit a curve (shown in red) that minimizes the least-square error $\sum_{i=1}^n (y_i - f(x_i))^2$. In this example, we see that the curve generated by regression is almost a perfect fit with a very small total loss value.

<img src="genetic_alg_fit1.png">

### Example 2

However, here is a different example, where the green 'x's denote the data and the green curve the exact unknown "ground truth" curve that generated the data. The red curve shows the result from regression. Here the fit is not great meaning that the loss value (least square error) is large.

<img src="genetic_alg_fit2.png">


## Already Implemented Functionality

Please read a description of important functions that have already been provided to help you focus on programming the genetic algorithm solver.

Note that you do not need to modify any of this code but it is important to understand what this code does for you and familiarize yourself with key functions described below.

### Symbolic Expressions

In this project, we represent functions as trees (they will be called _abstract syntax trees_ in your programming languages class).


__Example__ Consider the function $e^{\sin(2x)} - e^{\cos(3x)}$. The abstract syntax tree for this function is shown below:

<img src="abstract-syntax-tree1.png" width="40%">

Note that each node represents an arithmetic operation or a unary function application.

Please examine the file `symbolicExpressions.py` to get an idea of how these are implemented. However, note that we have taken care of all the functions you will likely need. If you need special functionality, you are welcome to post on piazza and discuss how to implement it with us.



### Generating Arbitrary Expressions at Random

In file `makeRandomExpressions.py` we have implemented an important function `generate_random_expr` that takes in a `depth` parameter, a list of identifier (strings) `lst_of_identifiers` and some parameters (see `geneticAlgParams.py`) that govern how random expressions are generated.

Go ahead and try this code out by running `python3 makeRandomExpressions.py`. If you examine the expressions so generated, you will see that some expressions are not going to be viable:

**Example: ** `(((-5.622601942609484 - -2.0 + (7.645577904681765 * 1.0)) * sqrt(-1.0) * (sin(y)/sin(-2.0))) + (((-2.0 * 0.5 * 1.0 * x * 0.5)/2.167715126398198 - -1.0) * ((x/x)/(-1.0 + -2.0 + -0.8951586596410355))))`

Notice that we are calling `sqrt(-1.0)` which will fail. We need a way to check if an expression is well-defined or "viable", perhaps eliminating ones that are not, like the example above.


### Checking if an Expression is Viable (Well-Defined)

Function `is_viable_expr(expr, lst_of_identifiers, params)` in file `fitnessAndValidityFunctions.py` will check if an expression is well defined by generating a lot of test points over a given range and checking if the function can be successfully evaluated. 


### Fitness Function

For a regression problem we wish to __minimize__ the sum of square errors over data. Most genetic algorithms are framed as __maximizing__ fitness. We have implemented a fitness function for you in `fitnessAndValidityFunctions.py` :

`compute_fitness(fun_expr, lst_of_identifiers, params)` given a function expression `fun_expr`, a list of identifiers `lst_of_identifiers` and the parameters `params`. This function calculates the __negation__ of least square error so that you can focus on maximizing it. 




###  CrossOver and Mutation Operators

Genetic algorithms require you to implement two operators: _crossover_ and _mutation_.


**Crossover Operator** Sometimes called _mating_, involves two expressions `e1` and `e2`. A crossover selects a subtree (subexpression) each of `e1` and `e2` at random and simply swaps them. the subtree of `e1` now becomes a subtree of `e2` and vice-versa.

<img src="crossover-illustration.png" width="40%">


The function `random_subtree_crossover(e1, e2, copy)` in file `crossOverOperators.py` implements a cross over operator for you by selecting subtrees at random. The two expressions `e1` and `e2` are the expressions to cross over. The flag copy is set to `True` if we do not want to mutate `e1` and `e2` but rather work off copies of these expressions. It is advisable to set this flag to `True`.

**Mutation Operator** Another operation is to mutate a given expression tree at random. We have implemented mutation operator:

`random_expression_mutation(e_orig, lst_of_identifiers, params, copy)`

This implements three different mutations: select a subexpression at random and replace it with a random expression; or replace the entire expression by the subexpression; or make the subexpression part of a random expression.

You do not need to modify this code.

## Task: Implement a Genetic Algorithm Solver 

File `geneticSearchAlgorithms.py` is the file you need to complete to finish the implementation of the genetic algorithm.

In this file we have a class `GASolver` with a function `run_ga_iterations(num_iters)` that you have to implement according to the description below.

## 1. Generate an Initial Population

GA maintains a population of candidate expressions and "evolves" this population. To start off, you need to 
create an initial population.  If you wish, you can store the current population as a list `self.pop` in the `GASolver` class. 

### TODO: Implement a method creating initial population
  - `self.N` denotes the population size. Your population must have N expressions.
  - Iterate until you have `self.N` viable expressions in your population (as described below):
    - Generate a expr using the function `generate_random_expr` (see description above). You can use `self.params.depth` for your expressions or hardcode a "reasonable" value such as "3" for depth.
    - Check if the expr generated is "viable" using the function `is_viable_expr` (see description above). 
    - If an expression is not viable, do not add it to the population. 
 
  


### 2. Each Iteration of Genetic Algorithm

In each iteration of the GA, we will build the "next generation" of expressions from the current population.
 - **Important** The size of the next generation will be the same (i.e, we do not grow or shrink population size).
 
There are two sources from which the next generation arises:

 - **Elitism** The top `k` members in terms of fitness of current population will be  moved on to next generation (i.e, elites get to live long).
 - **Mating (Crossover) and Mutation** We will perform $N-k$ pairwise crossovers followed by mutations to generate the remaining $N-k$ members.

<img src="mutation-crossover-next-gen.png" width=45%>


The figure above illustrates the overall process by which the next generation is created.

The genetic algorithm `params` class has a field `params.elitism_fraction` (currently set to 0.2). This fraction can be used to select `k` as (`elitism_fraction * self.N`). In this case, the top 20\% of the population will continue to live for one more generation.
 
 
### TODO: Implement Mutation/Crossover 

- Let `k` be the number of elites. We need to generate `N-k` expressions through mutations/crossover.

- Iterate the process below until we generate `N-k` expressions:
  - Select two expressions `e1` and `e2` from the __current population__ with the probability of selecting an element proportional to $e^{\text{fitness}/\text{temperature}}$. We have a parameter `params.temperature` that you can use. Unlike simulated annealing, the temperature does not change.
    - Note that it is important that expressions be chosen according to fitness to ensure that the fittest get to crossover. Larger the fitness, higher the probability of choosing. We encourage you to use the formula above.
    - There is a helpful function `random.choices` that allows you to choose a random element from a list and pass in a weight list of positive numbers (need not add up to 1). https://www.geeksforgeeks.org/how-to-get-weighted-random-choice-in-python/
  - Crossover `e1` and `e2` using the method provided for this purpose: `random_subtree_crossover`. Use `copy=True` and you will get a new pair of expressions `e1_cross` and `e2_cross` as a result.
  - Mutate `e1_cross` and `e2_cross` separately. Use the method provided for this purpose: `random_expression_mutation`.
  - Check whether they are viable using the method `is_viable_expr`. If yes, add the viable ones to the result, otherwise reject any expression that is not viable.
  - Keep doing this process until, we have collected the required number of expressions : $N-k$.
  
### TODO: Elitism

Sort the current population by descending order fitness and choose top-k elites.


### TODO: Next Generation Population
  
  The next generation is made up of k elites and N-k crossovers.  At the end of each iteration, do not forget to do the following: 
  - We need to maintain the best solution so far. Make sure to update these fields: `best_solution_so_far` and `best_fitness_so_far`.
  - Update the population. 
  - We have provided a field called `population_stats`. You should append the best fitness at each iteration to generate a  plot at the end that tracks how fitness changes across iterations.
  



 ### Testing your Work
 
 To test your work, you can run the python script `geneticSearchAlgorithms.py` and `curveFitting.py`.
 Unlike regular assignments, we cannot write test cases for a genetic algorithm.
 
  - Ensure that the tests run without crashing or any errors.
  - Ensure that your genetic algorithm increases the overall fitness from a small value to a value closer to 0 for each test in `curveFitting.py`.
  - Finally, try new examples to see if your algorithm seems to be working.
 

## Fields of GASolver Class

We have provided some fields of the GASolver Class in the file `geneticSearchAlgorithms.py`.
  - `self.params` : a set of parameters to drive the genetic algorithm and data from the curve fitting task. You will need to be aware of a few special parameters.
    - `self.params.depth`: The depth of the expression trees to be passed to the function `generate_random_expr` to create the initial population.
    - `self.params.temperature`: Used in the exponent to convert fitness values to probabilities of selecting the expression for crossover (see above).
    - `self.params.elitism_fraction`: The fraction of elites to be chosen for each population.
    - We suggest that you not modify other parameters that are needed by the mutation/crossover/random expression generation code we have provided.
 - `self.N`: the population size
 - `self.pop`: A list of expressions representing the current population.
 - `self.identifiers`: A list of identifiers (variable na,es).
 - `self.best_solution_so_far`: The best solution so far
 - `self.best_fitness_so_far`: The best fitness so far.
 - `self.population_stats`: A list of best fitness scores at each iteration to be used to generate plots. Please remember to maintain this field.
 
 You should feel free to add any other fields that you may need.
 
## Methods of GA Solver 

- `run_ga_iterations(self, num_iters)`: Run `num_iters` iterations of the genetic algorithm.
  - Should update the `best_solution_so_far`, `best_fitness_so_far` and `population_stats` fields

### Summary of Useful Methods

  - `generate_random_expr(depth, lst_of_identifiers, params)`: returns a randomly generated expression with `depth` as the suggested depth, `lst_of_identifiers` as the variables it can use and `params` as the parameters for various random actions. Implemented in file `makeRandomExpressions.py`.
  - `is_viable_expr(fun_expr, lst_of_identifiers, params)`: Tests a function expression at various points to make sure that it can be evaluated without errors. Implemented in file `fitnessAndValidityFunctions.py`
  - `compute_fitness(fun_expr, lst_of_identifiers, params)`: Compute a fitness score (negative number and larger fitness = better solution) for an expression. Implemented in file `fitnessAndValidityFunctions.py`
  - `random_subtree_crossover(e1, e2, copy = True)`: Crossover expressions `e1` and `e2`. Setting copy to True will not mutate e1 and e2. It is recommended to leave copy as True when calling this function. Returns a pair of expressions `(ea, eb)` after crossover.
  - `random_expression_mutation(e_orig, lst_of_identifiers, params, copy=True)`: mutate expression `e_orig` with list of identifiers and parameters. Setting copy as True will not mutate e_orig but instead return a mutated copy. Returns an expression.
  - `random.choices`:  allows you to choose a random element from a list and pass in a weight list of positive numbers (need not add up to 1). https://www.geeksforgeeks.org/how-to-get-weighted-random-choice-in-python/

## Submitting Your Work

Your submission must be a zip file uploaded to canvas.

  - You must submit all the files you have modified. You can modify any file __except__ `curveFitting.py`. Please leave that file unchanged since we will use it to help us evaluate your code.
  - Your submission must include a report in the form of a jupyter notebook: use the provided template `MiniProject1ReportTemplate.ipynb`, by answering the questions and benchmarking the solver's performance on the five problems provided. You can add your own functions to this list.

## Measuring Performance/Benchmarking

In the file `MiniProject1ReportTemplate.ipynb`, we have given you 5 benchmark problems. 
  - Attempt these problems only after you have finishedd and tested your implementation.
  - If you modify any of the python files, remember to "restart the kernel" or else the changes will not take effect.
  - Run each problem 5 different times and record the best solution/fitness for each trial.
  - Also record the running time.
  - Record your observations in the results table.
  
 Try your own benchmarks (as many as you can) and add them to the end of the report as well as to the table.


## Deliverables

Single zip file with 

- Python code (do not modify curveFitting.py) file.
- The `MiniProjectReportTemplate` notebook.


## Going "Over and Above"

- Successfully completing all portions of the project above will give you 90% points for the assignment (9/10).

If you want to impress us and earn 100%, then you can try at least one idea among the following lines:
 - Implement simulated annealing using the mutation operator. Compare Genetic Algorithms against simulated annealing.
 - Modify the mutation operator in a nontrivial manner to see if it helps/hurts performance.
 - Modify parameters like population size (set to 500 now) and number of iterations (set to 100) to measure their impacts on the benchmark problems. Add additional tables and write a small note.
 - Describe all you did and the results at the end of the report.


### That's All Folks!